In [1]:
import pandas as pd
import numpy as np
import cv2
import os
import h5py
from keras import backend as K
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.layers import Input
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
from matplotlib import pyplot as plt
from sklearn.model_selection import StratifiedKFold

Using TensorFlow backend.


In [2]:
val_set = pd.read_pickle('/external_drive/BUS_Deep_Learning/train_set_class_nn.pickle')
train_set = pd.read_pickle('/external_drive/BUS_Deep_Learning/val_set_class_nn.pickle')
input_data = pd.concat([train_set,val_set]).sample(frac=1)

In [3]:
model_save_dir ='models/classification_models/inception_resnet_v2-noaug_no-normals_histogram_eq-k_fold'
os.makedirs(model_save_dir,exist_ok=True)

In [4]:
image_shape = (128,128,1)
input_tensor = Input(image_shape)

In [5]:
#batch size
train_batch_size = 16
val_batch_size = 16
nepochs = 20
#model optimizer
optimizer = Adam(beta_1=0.99,beta_2=0.99,lr=1.0e-4,decay=1.0e-6)
#loss
loss= 'categorical_crossentropy'

In [6]:
def get_xy_data(fmatrix):
    x_input = []
    y_output = []
    y_kfold = []
    for index, row in fmatrix.iterrows():
        print('loading sequence : {}'.format(row['image_path']))
        img = cv2.imread(row['image_path'], cv2.IMREAD_GRAYSCALE)
        img = cv2.equalizeHist(img)
#         clahe = cv2.createCLAHE()
#         img = clahe.apply(img)
        img = cv2.resize(img,(128,128))
        img = img/255.0
        x_input.append(img.reshape(img.shape + (1,)))
        y_output.append(list(row['lesion_state_one_hot']))
        y_kfold.append(row['lesion_state'])
        
    return np.array(x_input), np.array(y_output), np.array(y_kfold)

In [7]:
X, Y, output = get_xy_data(input_data)

loading sequence : /external_drive/BUS_Deep_Learning/data/Dataset_BUSI_with_GT/benign/benign (202).png
loading sequence : /external_drive/BUS_Deep_Learning/data/Dataset_BUSI_with_GT/benign/benign (28).png
loading sequence : /external_drive/BUS_Deep_Learning/data/Dataset_BUSI_with_GT/benign/benign (242).png
loading sequence : /external_drive/BUS_Deep_Learning/data/Dataset_BUSI_with_GT/benign/benign (183).png
loading sequence : /external_drive/BUS_Deep_Learning/data/Dataset_BUSI_with_GT/benign/benign (32).png
loading sequence : /external_drive/BUS_Deep_Learning/data/Dataset_BUSI_with_GT/benign/benign (190).png
loading sequence : /external_drive/BUS_Deep_Learning/data/Dataset_BUSI_with_GT/benign/benign (194).png
loading sequence : /external_drive/BUS_Deep_Learning/data/Dataset_BUSI_with_GT/malignant/malignant (126).png
loading sequence : /external_drive/BUS_Deep_Learning/data/Dataset_BUSI_with_GT/malignant/malignant (76).png
loading sequence : /external_drive/BUS_Deep_Learning/data/Datase

loading sequence : /external_drive/BUS_Deep_Learning/data/Dataset_BUSI_with_GT/benign/benign (171).png
loading sequence : /external_drive/BUS_Deep_Learning/data/Dataset_BUSI_with_GT/benign/benign (261).png
loading sequence : /external_drive/BUS_Deep_Learning/data/Dataset_BUSI_with_GT/benign/benign (305).png
loading sequence : /external_drive/BUS_Deep_Learning/data/Dataset_BUSI_with_GT/benign/benign (298).png
loading sequence : /external_drive/BUS_Deep_Learning/data/Dataset_BUSI_with_GT/malignant/malignant (81).png
loading sequence : /external_drive/BUS_Deep_Learning/data/Dataset_BUSI_with_GT/malignant/malignant (135).png
loading sequence : /external_drive/BUS_Deep_Learning/data/Dataset_BUSI_with_GT/benign/benign (140).png
loading sequence : /external_drive/BUS_Deep_Learning/data/Dataset_BUSI_with_GT/malignant/malignant (67).png
loading sequence : /external_drive/BUS_Deep_Learning/data/Dataset_BUSI_with_GT/benign/benign (14).png
loading sequence : /external_drive/BUS_Deep_Learning/data/

loading sequence : /external_drive/BUS_Deep_Learning/data/Dataset_BUSI_with_GT/benign/benign (395).png
loading sequence : /external_drive/BUS_Deep_Learning/data/Dataset_BUSI_with_GT/benign/benign (17).png
loading sequence : /external_drive/BUS_Deep_Learning/data/Dataset_BUSI_with_GT/benign/benign (117).png
loading sequence : /external_drive/BUS_Deep_Learning/data/Dataset_BUSI_with_GT/malignant/malignant (97).png
loading sequence : /external_drive/BUS_Deep_Learning/data/Dataset_BUSI_with_GT/benign/benign (273).png
loading sequence : /external_drive/BUS_Deep_Learning/data/Dataset_BUSI_with_GT/benign/benign (128).png
loading sequence : /external_drive/BUS_Deep_Learning/data/Dataset_BUSI_with_GT/malignant/malignant (99).png
loading sequence : /external_drive/BUS_Deep_Learning/data/Dataset_BUSI_with_GT/malignant/malignant (14).png
loading sequence : /external_drive/BUS_Deep_Learning/data/Dataset_BUSI_with_GT/benign/benign (429).png
loading sequence : /external_drive/BUS_Deep_Learning/data/D

loading sequence : /external_drive/BUS_Deep_Learning/data/Dataset_BUSI_with_GT/benign/benign (350).png
loading sequence : /external_drive/BUS_Deep_Learning/data/Dataset_BUSI_with_GT/benign/benign (392).png
loading sequence : /external_drive/BUS_Deep_Learning/data/Dataset_BUSI_with_GT/malignant/malignant (196).png
loading sequence : /external_drive/BUS_Deep_Learning/data/Dataset_BUSI_with_GT/benign/benign (393).png
loading sequence : /external_drive/BUS_Deep_Learning/data/Dataset_BUSI_with_GT/benign/benign (270).png
loading sequence : /external_drive/BUS_Deep_Learning/data/Dataset_BUSI_with_GT/malignant/malignant (117).png
loading sequence : /external_drive/BUS_Deep_Learning/data/Dataset_BUSI_with_GT/benign/benign (302).png
loading sequence : /external_drive/BUS_Deep_Learning/data/Dataset_BUSI_with_GT/benign/benign (365).png
loading sequence : /external_drive/BUS_Deep_Learning/data/Dataset_BUSI_with_GT/benign/benign (327).png
loading sequence : /external_drive/BUS_Deep_Learning/data/Dat

loading sequence : /external_drive/BUS_Deep_Learning/data/Dataset_BUSI_with_GT/benign/benign (356).png
loading sequence : /external_drive/BUS_Deep_Learning/data/Dataset_BUSI_with_GT/benign/benign (280).png
loading sequence : /external_drive/BUS_Deep_Learning/data/Dataset_BUSI_with_GT/benign/benign (362).png
loading sequence : /external_drive/BUS_Deep_Learning/data/Dataset_BUSI_with_GT/malignant/malignant (148).png
loading sequence : /external_drive/BUS_Deep_Learning/data/Dataset_BUSI_with_GT/malignant/malignant (37).png
loading sequence : /external_drive/BUS_Deep_Learning/data/Dataset_BUSI_with_GT/benign/benign (81).png
loading sequence : /external_drive/BUS_Deep_Learning/data/Dataset_BUSI_with_GT/benign/benign (405).png
loading sequence : /external_drive/BUS_Deep_Learning/data/Dataset_BUSI_with_GT/malignant/malignant (129).png
loading sequence : /external_drive/BUS_Deep_Learning/data/Dataset_BUSI_with_GT/benign/benign (1).png
loading sequence : /external_drive/BUS_Deep_Learning/data/D

loading sequence : /external_drive/BUS_Deep_Learning/data/Dataset_BUSI_with_GT/malignant/malignant (90).png
loading sequence : /external_drive/BUS_Deep_Learning/data/Dataset_BUSI_with_GT/malignant/malignant (35).png
loading sequence : /external_drive/BUS_Deep_Learning/data/Dataset_BUSI_with_GT/benign/benign (70).png
loading sequence : /external_drive/BUS_Deep_Learning/data/Dataset_BUSI_with_GT/benign/benign (379).png
loading sequence : /external_drive/BUS_Deep_Learning/data/Dataset_BUSI_with_GT/benign/benign (251).png
loading sequence : /external_drive/BUS_Deep_Learning/data/Dataset_BUSI_with_GT/benign/benign (414).png
loading sequence : /external_drive/BUS_Deep_Learning/data/Dataset_BUSI_with_GT/benign/benign (220).png
loading sequence : /external_drive/BUS_Deep_Learning/data/Dataset_BUSI_with_GT/benign/benign (399).png
loading sequence : /external_drive/BUS_Deep_Learning/data/Dataset_BUSI_with_GT/benign/benign (213).png
loading sequence : /external_drive/BUS_Deep_Learning/data/Datase

loading sequence : /external_drive/BUS_Deep_Learning/data/Dataset_BUSI_with_GT/benign/benign (196).png
loading sequence : /external_drive/BUS_Deep_Learning/data/Dataset_BUSI_with_GT/benign/benign (36).png
loading sequence : /external_drive/BUS_Deep_Learning/data/Dataset_BUSI_with_GT/benign/benign (264).png
loading sequence : /external_drive/BUS_Deep_Learning/data/Dataset_BUSI_with_GT/malignant/malignant (62).png
loading sequence : /external_drive/BUS_Deep_Learning/data/Dataset_BUSI_with_GT/malignant/malignant (162).png
loading sequence : /external_drive/BUS_Deep_Learning/data/Dataset_BUSI_with_GT/benign/benign (137).png
loading sequence : /external_drive/BUS_Deep_Learning/data/Dataset_BUSI_with_GT/malignant/malignant (102).png
loading sequence : /external_drive/BUS_Deep_Learning/data/Dataset_BUSI_with_GT/benign/benign (37).png
loading sequence : /external_drive/BUS_Deep_Learning/data/Dataset_BUSI_with_GT/malignant/malignant (210).png
loading sequence : /external_drive/BUS_Deep_Learning

In [8]:
seed = 9
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)

In [9]:
counter = 0
for train, test in kfold.split(X, output):
    inceptionresnetv2 = InceptionResNetV2(input_tensor=input_tensor, weights=None, include_top=True, classes=2)
    inceptionresnetv2.compile(loss=loss, optimizer=optimizer, metrics = ['categorical_accuracy'])      
    #create checkpoint callback to save model on epochs that have best validation accuracy
    checkpoint = ModelCheckpoint(model_save_dir + '/model_{}'.format(str(counter))+ '_weights-improvement-{epoch:02d}-{val_categorical_accuracy:.2f}.hdf5', monitor='val_categorical_accuracy', verbose=1, save_best_only=True, mode='max')
    callbacks_list = [checkpoint]
    #steps per epoch
    train_steps_per_epoch = int(np.ceil(len(X[train])/train_batch_size))
    val_steps_per_epoch = int(np.ceil(len(X[test])/val_batch_size))
    #Generator for keras model
    data_gen_args = dict(width_shift_range=0.1,
                        height_shift_range=0.1,
                        shear_range=10,
                        horizontal_flip=True,
                        fill_mode='nearest')
    # train_image_datagen = ImageDataGenerator(**data_gen_args)   
    train_image_datagen = ImageDataGenerator() 
    val_image_datagen = ImageDataGenerator()   
    training_set_generator = train_image_datagen.flow(X[train], Y[train], batch_size=train_batch_size, shuffle=True,seed=seed)
    validation_set_generator = val_image_datagen.flow(X[test], Y[test], batch_size=val_batch_size, shuffle=True, seed=seed)
    history = inceptionresnetv2.fit_generator(training_set_generator, validation_data = validation_set_generator, steps_per_epoch=train_steps_per_epoch, validation_steps=val_steps_per_epoch, callbacks=callbacks_list, epochs = nepochs, verbose=True)
    counter +=1

Epoch 1/20
33/33 [==============================] - 74s 2s/step - loss: 0.7090 - categorical_accuracy: 0.6681 - val_loss: 1.0144 - val_categorical_accuracy: 0.7000

Epoch 00001: val_categorical_accuracy improved from -inf to 0.70000, saving model to models/classification_models/inception_resnet_v2-noaug_no-normals_histogram_eq-k_fold/model_0_weights-improvement-01-0.70.hdf5
Epoch 2/20
33/33 [==============================] - 20s 593ms/step - loss: 0.3056 - categorical_accuracy: 0.8969 - val_loss: 0.5743 - val_categorical_accuracy: 0.7692

Epoch 00002: val_categorical_accuracy improved from 0.70000 to 0.76923, saving model to models/classification_models/inception_resnet_v2-noaug_no-normals_histogram_eq-k_fold/model_0_weights-improvement-02-0.77.hdf5
Epoch 3/20
33/33 [==============================] - 20s 594ms/step - loss: 0.1004 - categorical_accuracy: 0.9659 - val_loss: 1.1374 - val_categorical_accuracy: 0.8000

Epoch 00003: val_categorical_accuracy improved from 0.76923 to 0.80000, 


Epoch 00009: val_categorical_accuracy did not improve from 0.83077
Epoch 10/20
33/33 [==============================] - 19s 590ms/step - loss: 0.0540 - categorical_accuracy: 0.9792 - val_loss: 1.0442 - val_categorical_accuracy: 0.7231

Epoch 00010: val_categorical_accuracy did not improve from 0.83077
Epoch 11/20
33/33 [==============================] - 20s 593ms/step - loss: 0.0653 - categorical_accuracy: 0.9792 - val_loss: 0.8732 - val_categorical_accuracy: 0.8154

Epoch 00011: val_categorical_accuracy did not improve from 0.83077
Epoch 12/20
33/33 [==============================] - 19s 591ms/step - loss: 0.0582 - categorical_accuracy: 0.9773 - val_loss: 0.8523 - val_categorical_accuracy: 0.8000

Epoch 00012: val_categorical_accuracy did not improve from 0.83077
Epoch 13/20
33/33 [==============================] - 20s 592ms/step - loss: 0.0424 - categorical_accuracy: 0.9886 - val_loss: 0.6870 - val_categorical_accuracy: 0.7769

Epoch 00013: val_categorical_accuracy did not improve f

Epoch 2/20
33/33 [==============================] - 20s 606ms/step - loss: 0.2344 - categorical_accuracy: 0.9268 - val_loss: 0.5517 - val_categorical_accuracy: 0.7287

Epoch 00002: val_categorical_accuracy did not improve from 0.73643
Epoch 3/20
33/33 [==============================] - 20s 602ms/step - loss: 0.0941 - categorical_accuracy: 0.9716 - val_loss: 0.9293 - val_categorical_accuracy: 0.6434

Epoch 00003: val_categorical_accuracy did not improve from 0.73643
Epoch 4/20
33/33 [==============================] - 20s 604ms/step - loss: 0.0898 - categorical_accuracy: 0.9716 - val_loss: 1.1188 - val_categorical_accuracy: 0.7674

Epoch 00004: val_categorical_accuracy improved from 0.73643 to 0.76744, saving model to models/classification_models/inception_resnet_v2-noaug_no-normals_histogram_eq-k_fold/model_3_weights-improvement-04-0.77.hdf5
Epoch 5/20
33/33 [==============================] - 20s 614ms/step - loss: 0.0508 - categorical_accuracy: 0.9848 - val_loss: 1.0136 - val_categoric


Epoch 00013: val_categorical_accuracy did not improve from 0.82171
Epoch 14/20
33/33 [==============================] - 20s 609ms/step - loss: 0.0509 - categorical_accuracy: 0.9735 - val_loss: 1.0347 - val_categorical_accuracy: 0.8062

Epoch 00014: val_categorical_accuracy did not improve from 0.82171
Epoch 15/20
33/33 [==============================] - 20s 609ms/step - loss: 0.0713 - categorical_accuracy: 0.9792 - val_loss: 1.1745 - val_categorical_accuracy: 0.8372

Epoch 00015: val_categorical_accuracy improved from 0.82171 to 0.83721, saving model to models/classification_models/inception_resnet_v2-noaug_no-normals_histogram_eq-k_fold/model_4_weights-improvement-15-0.84.hdf5
Epoch 16/20
33/33 [==============================] - 20s 613ms/step - loss: 0.0444 - categorical_accuracy: 0.9848 - val_loss: 1.8123 - val_categorical_accuracy: 0.7674

Epoch 00016: val_categorical_accuracy did not improve from 0.83721
Epoch 17/20
33/33 [==============================] - 20s 607ms/step - loss: 